# An AI agent learns to play tic-tac-toe part 4: visualising the Q table
*Where we visualise the Q table during the training process using plotnine and ffmpeg*


    ffmpeg -r 30 -f image2 -s 1920x1080 -i qtable_ep%06d.png -vcodec libx264 -crf 25  -pix_fmt yuv420p qtable.mp4